In [2]:
from General import functies as functies
from General import classes as classes
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
rho, m, r1, r2, L = sp.symbols('rho m r1 r2 L')
massadichtheidformule = classes.vergelijking(m / (np.pi*(r2**2 - r1**2) * L), [m, r1, r2, L])
#massadichtheid koperen buis van 1 m
massa_Cu1m = classes.datapunt(0.364, 0.001, m, 'U') #kg
straal_binnenrand = classes.datapunt(14e-3, 0.5e-3, r1, 'U') #meter
straal_buitenrand = classes.datapunt(15e-3, 0.5e-3, r2, 'U') #meter
lengte = classes.datapunt(100.8e-2, 0.2e-2, L, 'U') #meter
massadichtheid_Cu = massadichtheidformule.evaluate([massa_Cu1m, straal_binnenrand, straal_buitenrand, lengte], None, rho)
print(massadichtheid_Cu)

"""
Vermoedelijk heeft de producent gesjoemeld in de dikte van de buis waardoor de berekening niet uitkomt.
Gezien het gecontroleerd is door de Europese Unie gaan we ervan uit dat het aan de >99.90% zuiverheidsnorm voldoet.
"""


datapunt: rho = [3963.62885094759, 1619.12502459597, 'Normaal']
